In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from modules.util.util import importstr, run_shell_cmd, get_best_model
from modules.util.logconf import logging
log = logging.getLogger('nb')

## Train

This is a semi-automated process. The learning rate will need to be manually set based on the results of the learning rate finder on the training dataset

First set the folder location of the training data

In [ ]:
train_data_path = 'COVID-19-20_v2/Train'     # replace with data path here

Then generate a metadata file and prepare a cache of preprocessed data. This will create a metadata/df_meta.fth file as well as a cache/ folder in the current directory.

In [ ]:
run_shell_cmd(log, 'prepcache.CovidPrepCacheApp', f'--data-path={train_data_path}', 
              f'--num-workers={4}', f'--batch-size={20}')

Use lr finder to find a good learning rate. This will run the model through a hundered batches which take a few minutes to complete on the gpu.

In [ ]:
run_shell_cmd(log, 'modules.util.lr_finder.LearningRateFinder')

Set the value of learning rate as found by the learning rate finder

In [ ]:
learning_rate = 1e-2

Begin training. Validation will run every 5 epochs and a copy of the model parameters will be saved under saved-models/ folder. The best model based on F1 score is saved with the prefix `.best.state`

In [ ]:
run_shell_cmd(log, 'training.CovidSegmentationTrainingApp', f'--lr={learning_rate}')

## Inference

In [ ]:
inference_data_path = 'COVID-19-20_TestSet'    # replace with inference data path here

Once again, we prepare the metadata and cache files. Note that this will overwrite the metadatafile from the train set

In [ ]:
run_shell_cmd(log, 'prepcache.CovidPrepCacheApp', f'--data-path={inference_data_path}', 
              f'--num-workers={4}', f'--batch-size={20}')

You can specify a UID from a CT scan based on the number found in the filename.
For example, pass in `0180_0` for `volume-covid19-A-0180_0_ct.nii.gz`

In [ ]:
run_shell_cmd(log, 'inference.CovidInferenceApp', '004', f'--data-path={inference_data_path}', 
              f'--model-path={model_path}')

If you wish to explicitly specify a model path, just pass in a `--model-flag="model path"` flag

In [ ]:
# model_path = get_best_model('saved-models', '2020-12-08_19.15.04.best.state') # params = model_folder, model_file_name
# run_shell_cmd(log, 'inference.CovidInferenceApp', '004', f'--data-path={inference_data_path}', f'--model-path={model_path}')

To run inference on all of the files, simply pass the `--run-all` flag as shown below

In [ ]:
run_shell_cmd(log, 'inference.CovidInferenceApp', f'--data-path={inference_data_path}', 
              f'--model-path={model_path}', '--run-all')